In [1]:
!pip install xmltodict

In [3]:
import xmltodict
import datetime

class medicalDocument:
    def __init__(self):
        self.age = None
        self.gender = None  # 1-М, 2-Ж
        self.mainDisease = None  # Код заболевания
        self.instrumentalText = None
        self.laboratoryText = None
        self.morphologyText = None
        self.consultationText = None
        self.drugText = None
        self.nonDrugText = None
        self.surgicalText = None
        self.dietText = None
        self.treatText = None
        self.workText = None

    def parseXML(self, filePath):
        f = open(filePath, 'r', encoding='utf-8')
        xml = f.read()
        d = xmltodict.parse(xml)

        self.age = datetime.date.today().year - datetime.datetime.strptime(
            d['ClinicalDocument']['recordTarget']['patientRole']['patient']['birthTime']['@value'],'%Y%m%d').year
        self.gender = int(
            d['ClinicalDocument']['recordTarget']['patientRole']['patient']['administrativeGenderCode']['@code'])

        sections = d['ClinicalDocument']['component']['structuredBody']['component']

        for s in sections:
            self.parseSection(s['section'])

    def parseSection(self, section):
        code = section['code']['@code']
        match code:
            case 'HOSP':
                self.parseHOSP(section)
            case _:
                if 'component' not in section:
                    return

                self.parseComponentsText(section['component'])

    def parseHOSP(self, section):
        #ОБЩИЕ ДАННЫЕ О ГОСПИТАЛИЗАЦИИ
        entry = None

        # Может быть целый список entry, из них нужно найти подходящий
        for e in section['entry']:
            if 'act' not in e:
                continue

            if e['act']['@classCode'] == 'ACT':
                entry = e
                break

        if entry is None:
            return

        self.mainDisease = entry['act']['entryRelationship']['observation']['value']['@code']

    def parseComponentsText(self, section):
        for c in section:
            text = c['section']['text']
            code = c['section']['code']['@code']

            match code:
                case 'RESINSTR': self.instrumentalText = text
                case 'RESLAB': self.laboratoryText = text
                case 'RESMOR': self.morphologyText = text
                case 'RESCONS': self.consultationText = text
                case 'DRUG': self.drugText = text
                case 'NONDRUG': self.nonDrugText = text
                case 'SUR': self.surgicalText = text
                case 'RECDIET': self.dietText = text
                case 'RECTREAT': self.treatText = text
                case 'RECWORK': self.workText = text

    def printInfo(self):
        print()
        print('Данные пациента:')
        print('Возраст: {0} лет'.format(self.age))
        print('Пол: {0}'.format({1: 'Муж', 2: 'Жен'}[self.gender]))
        print('Осн. заболевание: {0}'.format(self.mainDisease))
        print('Инструментальные исследования:\n\t{0}'.format(self.instrumentalText))
        print('Лабораторные исследования:\n\t{0}'.format(self.laboratoryText))
        print('Морфологические исследования:\n\t{0}'.format(self.morphologyText))
        print('Консультация врача:\n\t{0}'.format(self.consultationText))
        print('Медикаментозное лечение:\n\t{0}'.format(self.drugText))
        print('Немедикаментозное лечение:\n\t{0}'.format(self.nonDrugText))
        print('Хирургическое вмешательство:\n\t{0}'.format(self.surgicalText))
        print('Диета:\n\t{0}'.format(self.dietText))
        print('Лечение:\n\t{0}'.format(self.treatText))
        print('Трудовые рекомендации:\n\t{0}'.format(self.workText))


doc = medicalDocument()

doc.parseXML('/content/medicalDocumentХОБЛ.xml')
doc.printInfo()


Данные пациента:
Возраст: 62 лет
Пол: Муж
Осн. заболевание: J44.1
Инструментальные исследования:
	Компьютерная томография одной анатомической области у взрослых (без контрастирования) от 15.03.2022: В спиральном режиме произведено исследование лёгких, средостения (исследование предоставлено на внешнем носителе). Паренхима лёгких неравномерной воздушности за счет интерстициальных изменений, участков линейного и тяжистого фиброза, инфильтративных полей не определяется. Бронхи прослеживаются до субсегментов, обычного калибра. Синусы свободны. Прослеживается высокое стояние левого купола диафрагмы. В средостении дополнительных образований, увеличенных л/узлов не определяется. В аорте, коронарных артериях кальцинированные атеросклеротические бляшки. Заключение: КТ-картина интерстициальных, фиброзных изменений легких. Высокое стояние левого купола диафрагмы. Аортокоронаросклероз.Тест с 6-минутной ходьбой 17.03.2022 : пройденное расстояние 245 метров, исходно SpO2 90%, Borg 3, в конце spO2 8

In [4]:
data = doc.laboratoryText

In [56]:
data1 = re.split(r'КЩС\sот\s[0-9][0-9].[0-9][0-9].[0-9][0-9][0-9][0-9]:', data)
#data2 = re.split("Исследование", data)
data2 = re.findall(r'([А-ЯЁа-яё]+)\sот\s', data)

datas = {
    'HGB': re.findall(r'HGB:\s([\w,]+)\s', data),
    'Глюкоза': re.findall(r'Глюкоза:\s([\w,]+)\s', data),
    'Белок общий': re.findall(r'Глюкоза:\s(\w+)\s', data),
}

datas

{'HGB': ['173', '168,3'],
 'Глюкоза': ['5,4', '15,4', '7,3', '6,2', '5,9', 'Отрицательно'],
 'Белок общий': ['Отрицательно']}

In [ ]:
import re
import json
words = re.split("[;:]", data)
len(words)
words

In [ ]:
df = pd.DataFrame(words)
df

In [8]:
from json import loads, dumps
import pandas as pd


df = df.reset_index(drop=True)
result = df.to_json(orient="split")
parsed = loads(result)
parsed.pop("index")
parsed.pop("columns")
dataset = json.dumps(parsed, indent=4, sort_keys=False, ensure_ascii=False)
with open('dataset.json', 'w') as f:
    f.write(dataset)